In [22]:
import os
aipt_dir = '/home/ec2-user/SageMaker/antibody-in-pytorch/'
model_dir = 'AIPT/Models/Beshnova2020'
model_dir_abs = os.path.join(aipt_dir, model_dir)
os.chdir(model_dir_abs)
%pwd

'/home/ec2-user/SageMaker/antibody-in-pytorch/AIPT/Models/Beshnova2020'

In [2]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch
from AIPT.Utils.logging import today, current_time

In [3]:
sagemaker_session = sagemaker.Session()

In [4]:
# log_root_dir = '/home/ec2-user/SageMaker/logs/tensorboard'
run_root_dir = 's3://gv20interns/roger/runs'
run_name = 's3_test'
timezone = 'EST'
run_dir = os.path.join(run_root_dir, today(tz=timezone), run_name, current_time(tz=timezone))
out_dir = os.path.join(run_dir, 'out')
para_dict = {
    'seq_len': 12,
    'embedding_dim': 15,  # paper uses dim 15 PCA features
    'epoch': 1000,
    'learning_rate': 10 ** -4,
    'run_name': run_name,
    'run_dir': run_dir,
    'work_path': '/opt/ml/model',
}


In [33]:
pytorch_estimator = PyTorch(
    base_job_name="roger-beshnova2020--",
    entry_point=os.path.join(model_dir, 'train_job.py'),
    source_dir="../../../",
    role=get_execution_role(),
    output_path=out_dir,
    train_instance_type="ml.m5.large",
    train_instance_count=1,
    train_use_spot_instances=True,
    train_max_run=24 * 60 * 60,
    train_max_wait=24 * 60 * 60,
    py_version="py3",
    framework_version="1.6.0",
    hyperparameters=para_dict,
)

In [34]:
pytorch_estimator.fit({'data': 's3://gv20interns/OAS_dataset'})

2020-10-25 05:53:24 Starting - Starting the training job...
2020-10-25 05:53:26 Starting - Launching requested ML instances......
2020-10-25 05:54:35 Starting - Preparing the instances for training......
2020-10-25 05:55:28 Downloading - Downloading input data............
2020-10-25 05:57:51 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-10-25 05:57:51,701 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2020-10-25 05:57:51,703 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-10-25 05:57:51,712 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-10-25 05:57:53,176 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-10-25 05:57:54,536 sagemaker-training-toolkit INFO     Installing module with the foll

UnexpectedStatusException: Error for Training job roger-beshnova2020-2020-10-25-05-53-16-577: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/opt/conda/bin/python -m AIPT/Models/Beshnova2020/train_job --embedding_dim 15 --epoch 1000 --learning_rate 0.0001 --run_dir s3://gv20interns/roger/runs/2020-10-25/s3_test/00.36.04 --run_name s3_test --seq_len 12 --work_path /opt/ml/model"
/opt/conda/bin/python: No module named AIPT/Models/Beshnova2020/train_job

In [28]:
os.listdir('../../..')
os.listdir(os.path.join('../../..', model_dir))

['__pycache__',
 'work',
 'Replicate Liu2019.ipynb',
 '.ipynb_checkpoints',
 '__init__.py',
 'train_job.py',
 'pca_embedding.py',
 'Replicate_Beshnova2020.ipynb',
 'CNN.py',
 'figures',
 'model.py',
 'AAidx_PCA.txt',
 'CNNx1_classifier.py',
 'submit_train_job.ipynb',
 'OAS_index.txt']